In [ ]:
%%bash
ogr2ogr -f GPKG -nln rdcl -oo ENCODING=CP932 tokyo_west.gpkg ../../../_data/gsi_nummap/533945/DKG-SHP-533945-RdCL-20180829-0001.shp
ogr2ogr -f GPKG -update -append -nln rdedg -oo ENCODING=CP932 tokyo_west.gpkg ../../../_data/gsi_nummap/533945/DKG-SHP-533945-RdEdg-20180829-0001.shp 

In [ ]:
import geopandas as gpd
import pandas as pd

# create mcdonalds.json by web developer tools with company site
df = pd.read_json('./data/mcdonalds_231104.json')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf = gdf.drop(['condition_values'], axis=1)
gdf.to_file('./data/mcdonalds_231104.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

# made tour_of_mcd.geojson on QGIS with selsman ploblem plugin
mcd_df = gpd.read_file('./data/tour_of_mcd.geojson')
geoms = mcd_df.geometry
ngeoms = [geoms[0]]
for i in range(len(geoms) - 1):
    c0, c1 = ngeoms[i].coords[0], ngeoms[i].coords[-1]
    n0, n1 = geoms[i+1].coords[0], geoms[i+1].coords[-1]
    if c1 == n0:
        ngeoms.append(geoms[i+1])
    elif c1 == n1:
        ngeoms.append(geoms[i+1].reverse())
    else:
        assert False, (i, c0, c1, n0, n1)
route = LineString(sum([g.coords[:] for g in ngeoms], []))
gpd.GeoDataFrame({'geometry': [route]}).to_file('_route.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

route = gpd.read_file('_route.geojson').geometry[0]

In [ ]:
from pyproj import Transformer
import math

def get_camera_params(route, pitch=-40, the_range=300, step_meter=100, front_pad=10):
    def get_heading(p, q):
        p, q = p.coords[0], q.coords[0]
        dx, dy = q[0] - p[0], q[1] - p[1]
        r = math.atan2(dy, dx) * 180 / math.pi
        return int(round(90 - r))

    transformer = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True)
    rtransformer = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True)

    m_route = LineString(transformer.itransform(route.coords))
    distances = [l for l in range(20, int(m_route.length), step_meter)]
    locations = list(rtransformer.itransform([m_route.interpolate(l).coords[0] for l in distances]))
    headings = [get_heading(m_route.interpolate(l), m_route.interpolate(l + front_pad)) for l in distances]
    # return [in zip(locations, headings)]
    return [(lon, lat, heading, pitch, the_range) for (lon, lat), heading in zip(locations, headings)]

camera_params = get_camera_params(route, -10, the_range=25)

In [ ]:
from os import makedirs
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_driver_path = r'/Users/suika/Projects/GPTxGIS/codes/capturer/chromedriver-mac-arm64/chromedriver'

service = Service(chrome_driver_path)
options = Options()
# options.add_argument("--headless") # ヘッドレスモードオプションの例
driver = webdriver.Chrome(service=service, options=options)

scene = 'mcd_shinjuku'
makedirs(f'_batch_analyzed/{scene}/src_images', exist_ok=True)
makedirs(f'_batch_analyzed/{scene}/json', exist_ok=True)
for i, (lon, lat, h, p, r) in enumerate(camera_params):
    url = f'http://localhost:5173/?lat={lat}&lon={lon}&heading={h}&pitch={p}&range={r}'
    driver.get(url)
    time.sleep(10)
    fpath = f'_batch_analyzed/{scene}/src_images/{scene}_{p}_{r}_{i}.png'
    driver.save_screenshot(fpath)
    r = requests.post(
        'http://127.0.0.1:5000/analyze', 
        files={'file': open(fpath,'rb')}, 
        data={'lon': lon, 'lat': lat, 'heading': h, 'pitch': p, 'range': r}
    )
driver.quit()

In [ ]:
[1,2] + [3]